In [ ]:
# A IMPORTER

import pandas as pd
import json
import unidecode
import re
import numpy as np

## Cleaning the data

On charge notre dataset. Puis on créé un autre dataset reprenant les features qui nous intéresse pour appliquer nos processus NLP.

In [ ]:
data = pd.read_json('indeed6.json', encoding = 'utf-8', lines=True)

In [ ]:
job_to_class = data[['titre', 'texte']]

Ici, nous avons décidé d'utiliser la libraire spaCy. C'est elle qui nous permettra de parseriser nos descriptions et nos titres, retirer les stop words, et tokenizer.

In [ ]:
import spacy 
#Chargeons notre vocabulaire spaCy
nlp_fr = spacy.load("fr_core_news_sm")

In [ ]:
#Chargeons nos stopwords qu'on mettra dans une liste
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.fr.stop_words import STOP_WORDS
french_stopwords = list(spacy.lang.fr.stop_words.STOP_WORDS)
english_stopwords = list(spacy.lang.en.stop_words.STOP_WORDS)
stop_words = french_stopwords + english_stopwords

#J'importe la ponctuation qui nous servira au traitement du texte
import string
punctuations = string.punctuation

#Ainsi que notre Parser qui comprend la structure grammatical de notre texte
from spacy.lang.fr import French
parser = French()

Dans un premier temps, on avait opté pour utiliser strictement spaCy pour le traitement global. Toutefois, il restait imparfait avec beaucoup de "déchet" dans nos tokens.
On choisit donc de subdiviser à nouveau notre traitement. 
Tout d'abord on retire les caractères indésirables. Puis, dans la mesure où on a déjà créé des nouvelles features grâce au regex, on supprime ces éléments de nos données afin d'éviter tout risque de colinéarité dans l'entraînement de nos modèles de machine learning.
Enfin on tokenize nos textes et on nettoie à nouveau les caractères indésirables qui ont pu se glisser dedans.

In [ ]:
from string import digits

#Ces caractères indésirables ne nous fournissent pas d'information pertinente. On les retire
def clean_and_lower(string):
    forbid_car= [":", ";", ",", "&", "(", ")",
                '"', "!", "?", "*", "-", "\n", 
                "...", "/","'"]
    for car in forbid_car:
            cleaned_string = string.replace(car, ' ')
    return cleaned_string

#On retire les termes déjà étudié lors de nos regex, et qui sont déjà des features
def terme_a_retirer(string):
        string = re.sub(pattern = '(jeune[es().]* diplôme[es().]*|junior[s]*|novice[s]*| débutant[es().]*)', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = '(expérimenté[es().]*|senior[s]*|confirmé[es().]*)', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = '(licence|bac[ +]*[23/]+|iut|dut|bts| bsc|master|bac[ +]*5|ingénieur[.()es]*|grande[s]* école[s]*| msc|doctorat[s]*|docteur[.()es]*|ph[.]*d|thèse[s]*|bac)', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = '(minimum [1-9]*[/aou-]*\d+ an[nées]* d\'expérience|[1-9]*[/aou-]*\d+ an[nées]* d\'expérience|[1-9]*[/aou-]*\d+ an[nées]* minimum d\'expérience|expérience minimum de *[1-9]*[/aou-]*\d+ an[nees]|expérience|d\'expérience)', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = '(nantes|bordeaux|paris|île-de-france|lyon|toulouse)', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = '[\d+ ]*[ \-k€]*[\d+ ]*\d+,?\d+[ ]*[k€$]+', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = '(cdi|CDI|Cdi|cdd|CDD|Cdd|stage[s]*|stagiaire[s]*|intern[s]*|internship[s]*|freelance|Freelance|FREELANCE||indépendant[s]*|par an|/an|temps plein|3dexperience)', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = '(h[/ \-]*f|f[/ \-]*h)', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = '(salaire|[\d+]+[ ]*an[nées]*|[\d+]+[ ]*jour[s]|/mois|/jour[s]*|/semaine[s]*|[\d+]+ mois|[\d+]+ semaine[s]*|mois|semaine[s]*|jour[s]*])', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = '[\d+]+[]*[èe]*[mes]*', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = r'\d+', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = '[\w\.-]+@[\w\.-]+', repl = '', string = string, flags=re.IGNORECASE)
        return string

#On tokenize notre texte, on retire les pronoms, stop words, et on réduit chaque terme à sa racine
def tokenize(string):
    parsed_string = parser(string)
    tokenized_string = [word.lemma_.lower().strip() if word.lemma_ != '-PRON-' else word.lower_ for word in parsed_string]
    cleaned_tokenized_string = [word for word in tokenized_string if word not in punctuations and word not in stop_words]
    return cleaned_tokenized_string

#On retire la ponctuaction indésirable qui peut rester dans nos tokens
def strip_final_punctuation(list_of_tokens):
    forbid_car= [":", ";", ",", "&", "(", ")",
                '"', "!", "?", "*", "-", 
                "...", "/","'", "\\", "·", ".", "∙", "•"
                "–"]
    
    for i in range(len(list_of_tokens)):
        for car in forbid_car:
            list_of_tokens[i] = list_of_tokens[i].replace(car, '')
    return list_of_tokens
    


On définit ensuite une fonction finale de nettoyage qui tournera sur nos descriptions et sur nos titres.

In [ ]:
def cleaning(string):
    cleaned_string = clean_and_lower(string)
    cleaned_string = terme_a_retirer(cleaned_string)
    tokenized_string = tokenize(cleaned_string)
    tokenized_string = strip_final_punctuation(tokenized_string)
    return tokenized_string
    

## Traitement des descriptions avec SpaCy

### I. Tokenize, clean, lemmatize

In [ ]:
#On tokenize complètement avec spaCy nos descriptions

job_to_class['spacy_description'] = [cleaning(job_to_class.loc[i, 'texte']) for i in range(len(job_to_class['texte']))]

### II. Create the TF-IDF term-documents matrix with the processed data

Il nous faut maintenant évaluer le poid relatif de chaque terme dans nos descriptions. On choisit donc de vectoriser en utilisant une matrice Tf-Idf.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

L'initialisation des paramètres de notre vectorisation reste encore une question ouverte. L'idéal aurait été de tester automatiquement plusieurs nombre maximal de features et plusieurs n_gram, les stocker dans un log, et choisir le plus adapté. Faute de temps, on a choisi un grand nombre de features.

In [ ]:
#On initialise nos paramètres pour la vectorisation

max_features_description = 5000
min_n_description = 1
max_n_description = 3

#On crée une fonction inutile. En effet TfidfVectorizer ne peux pas gérer des documents déjà tokenisés

def dummy_fun(doc): 
    return doc

#On vectorise

tfidf_vectorizer_description = TfidfVectorizer( analyzer = 'word',
                                    tokenizer = dummy_fun,
                                    preprocessor = dummy_fun,
                                    token_pattern = None,
                                    max_df = 1.0, min_df = 1,
                                    max_features = max_features_description, sublinear_tf = True,
                                    ngram_range=(min_n_description, max_n_description))


tfidf_matrix_description = tfidf_vectorizer_description.fit_transform(job_to_class['spacy_description'])


### III. Get the features

On utilise maintenant notre matrice term-document pour extraire les features les plus importantes de nos descriptions. C'est celles-ci qui vont ensuite nous servir pour l'extraction de topic.

In [ ]:
features_description = tfidf_vectorizer_description.get_feature_names()

In [ ]:
print(features_description)

### IV. Find the (features x topics) matric with the LDA/LSI/NMF and generate groups


Une fois nos features extraites, on choisit d'utiliser trois méthodes pour extraire les topics: Nonnegative Matrix Factorisation, Latent Dirichlet Allocation et Latent Semantic Index.
On créé nos trois matrices d'extraction.

In [ ]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD

In [ ]:
lda_description = LatentDirichletAllocation(n_components= 10, max_iter=10, learning_method='online',verbose=True)
data_lda_description = lda_description.fit_transform(tfidf_matrix_description)

In [ ]:
# Non-Negative Matrix Factorization Model
nmf_description = NMF(n_components = 10)
data_nmf_description = nmf_description.fit_transform(tfidf_matrix_description)

In [ ]:
# Latent Semantic Indexing Model using Truncated SVD
lsi_description = TruncatedSVD(n_components = 10)
data_lsi_description = lsi_description.fit_transform(tfidf_matrix_description)

On imprime nos divers topics pour observer s'il y a de la cohérence et si c'est exploitable.

In [ ]:
# Functions for printing keywords for each topic
def selected_topics(model, top_n = 10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(tfidf_vectorizer_description.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])

In [ ]:
print("LDA_description Model:")
selected_topics(lda_description)

In [ ]:
print("LSI_description Model:")
selected_topics(lsi_description)

In [ ]:
print("NMF_description Model:")
selected_topics(nmf_description)

## Traitement des titres avec SpaCy

Ce qui a été obtenu via le traitement des descriptions n'est finalement pas satisfaisant. On essaye ici de concentrer le traitement sur les titres de chaque annonce. L'intuition c'est que l'information pour clusteriser nos annonces est plus concentrée sur les titres. Et donc la possibilité de différenciation sera plus importante.

Les étapes étant les mêmes que pour la description, on ne répétera pas les commentaires précédents. On se contentera d'analyser les résultats.

### I. Tokenize, clean, lemmatize

In [ ]:
job_to_class['spacy_titre'] = [cleaning(job_to_class.loc[i, 'titre']) for i in range(len(job_to_class['titre']))]

### II. Create the TF-IDF term-documents matrix with the processed data

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

Ici on a fait le choix de réduire les max_features à 500, et le n_gram à 1. En effet, les titres sont bien plus court, et bien plus redondants. Mais là aussi il aurait fallu tester automatiquement plusieurs valeurs et sélectionner celles qui donne les meilleures métriques.

In [ ]:
#On initialise nos paramètres pour la vectorisation

max_features_titres = 500
min_n_titres = 1
max_n_titres = 1

#On crée une fonction inutile. En effet TfidfVectorizer ne peux pas gérer des documents déjà tokenisés

def dummy_fun(doc): 
    return doc

#On vectorise

tfidf_vectorizer_titres = TfidfVectorizer( analyzer = 'word',
                                    tokenizer = dummy_fun,
                                    preprocessor = dummy_fun,
                                    token_pattern = None,
                                    max_df = 1.0, min_df = 1,
                                    max_features = max_features_titres, sublinear_tf = True,
                                    ngram_range=(min_n_titres, max_n_titres))


tfidf_matrix_titres = tfidf_vectorizer_titres.fit_transform(job_to_class['spacy_titre'])


### III. Get the features !

In [ ]:
features_titres = tfidf_vectorizer_titres.get_feature_names()
features_titres

### IV. Find the (features x topics) matric with the LDA/LSI/NMF and generate groups

In [ ]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD

In [ ]:
lda_titres = LatentDirichletAllocation(n_components= 20, max_iter=10, learning_method='online',verbose=True)
data_lda_titres = lda_titres.fit_transform(tfidf_matrix_titres)

In [ ]:
df_topic_20 = pd.DataFrame(data_lda_titres)
df_topic_20
df_topic_20.to_csv("topic_20.csv")

In [ ]:
# Non-Negative Matrix Factorization Model
nmf_titres = NMF(n_components = 20)
data_nmf_titres = nmf_titres.fit_transform(tfidf_matrix_titres)

In [ ]:
# Latent Semantic Indexing Model using Truncated SVD
lsi_titres = TruncatedSVD(n_components = 15)
data_lsi_titres = lsi_titres.fit_transform(tfidf_matrix_titres)

In [ ]:
# Functions for printing keywords for each topic
def selected_topics(model, top_n = 15):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(tfidf_vectorizer_titres.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])

In [ ]:
# print("LDA_titres Model:")
# type(selected_topics(lda_titres))

In [ ]:
# print("LSI_titres Model:")
# selected_topics(lsi_titres)

In [ ]:
print("NMF_titres Model:")
print(type(selected_topics(nmf_titres)))

In [ ]:
df_topic_20 = pd.DataFrame(data_nmf_titres)
df_topic_20.to_csv("topic_20.csv")

In [ ]:
df_topic_20.describe()

## Conclusion

L'application de notre méthode de NLP sur les titres s'avère donner des extractions de topic plus cohérent que celle sur les descriptions. Toutefois l'ajout de notre matrice NMF dans les features de notre dataset nous fait gagner une accuracy relativement marginale. Et malheureusement les topics ne sont pas suffisamment cohérent pour alimenter la data analysis côté client.
Peut-être que la libraire Bert pourrait s'avérer plus utile sur ces deux niveaux.